In [1]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import *
from quantfreedom.simulate import run_df_backtest, or_backtest
from quantfreedom.strategies.strategy import Strategy


mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [2]:
candles = mufex_main.get_candles(symbol="BTCUSDT", timeframe="5m", since_date_ms=1699660800000, candles_to_dl=1200)

In [3]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)

In [4]:
# backtest_settings = BacktestSettings(
#     array_size=200,
#     gains_pct_filter=-90,
#     total_trade_filter=10,
#     upside_filter=-np.inf,
# )
backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([5]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2.5]),
    sl_based_on_add_pct=np.array([0.05, .1, .15]),
    sl_based_on_lookback=np.array([20]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5]),
    trail_sl_when_pct=np.array([1]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    logger_bool=True,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type=TakeProfitFeeType.Limit,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

In [5]:
strategy = Strategy(
    rsi_is_below=np.array([60]),
    rsi_length=np.array([14]),
    long_short="long",
)

In [6]:
run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    static_os=static_os,
    strategy=strategy,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 1
Total order settings to test: 3
Total combinations of settings to test: 3
Total candles: 1,200
Total candles to test: 3,600


,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,to_the_upside,fees_paid,total_pnl,ending_eq
0,0,0,26.0,3,23,-89.032,11.538,-0.916,442.605,-890.318,109.682
1,0,1,22.0,3,19,-85.683,13.636,-0.915,366.320,-856.826,143.174
2,0,2,18.0,3,15,-78.739,16.667,-0.940,315.964,-787.389,212.611


In [7]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    static_os=static_os,
    strategy=strategy,
    dos_index=2,
    ind_set_index=0,
    plot_results=True,
)

In [8]:
order_records_df

,ind_set_idx,or_set_idx,bar_idx,timestamp,datetime,order_status,equity,available_balance,cash_borrowed,cash_used,...,entry_size_usd,entry_price,exit_price,position_size_asset,position_size_usd,realized_pnl,sl_pct,sl_price,tp_pct,tp_price
0,0,2,50,1699675800000,2023-11-11 04:10:00,EntryFilled,1000.000,947.455,4480.825,52.545,...,4533.370,37101.3,NaN,0.122,4533.370,NaN,0.5,36922.4,1.8,37759.8
1,0,2,51,1699676100000,2023-11-11 04:15:00,EntryFilled,1000.000,840.685,13589.636,159.315,...,4734.756,37090.8,NaN,0.250,9268.126,NaN,0.5,36922.4,1.7,37742.8
2,0,2,52,1699676400000,2023-11-11 04:20:00,EntryFilled,1000.000,653.814,32143.769,346.186,...,9632.193,37070.7,NaN,0.510,18900.319,NaN,0.3,36974.0,1.3,37574.8
3,0,2,53,1699676700000,2023-11-11 04:25:00,EntryFilled,1000.000,409.133,56082.717,590.867,...,5629.496,37015.2,NaN,0.662,24529.815,NaN,0.3,36957.0,1.3,37573.0
4,0,2,53,1699676700000,2023-11-11 04:25:00,StopLossFilled,877.842,877.842,NaN,NaN,...,NaN,NaN,36957.0,NaN,NaN,-122.158,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,0,2,1172,1700012400000,2023-11-15 01:40:00,StopLossFilled,212.611,212.611,NaN,NaN,...,NaN,NaN,35474.0,NaN,NaN,6.681,NaN,NaN,NaN,NaN
92,0,2,1173,1700012700000,2023-11-15 01:45:00,EntryFilled,212.611,198.491,1625.813,14.120,...,1639.933,35477.0,NaN,0.046,1639.933,NaN,0.2,35411.0,1.0,35844.2
93,0,2,1174,1700013000000,2023-11-15 01:50:00,EntryFilled,212.611,173.810,4163.906,38.801,...,936.961,35472.7,NaN,0.072,2576.894,NaN,0.3,35374.0,1.3,35931.6
94,0,2,1175,1700013300000,2023-11-15 01:55:00,EntryFilled,212.611,136.545,7971.483,76.066,...,1306.749,35472.2,NaN,0.109,3883.643,NaN,0.3,35374.0,1.3,35928.8
